<a href="https://colab.research.google.com/github/guilpassos/Fuctura_Data_Science/blob/main/Arvore_Decisao_analiseClienteEcommerce_funcoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Análise de dados para aumentar nível de satisfação dos clientes de um e-commerce utilizando árvore de decisão.**

__________________________________________________




**Importação das bibliotecas Pandas e Numpy**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Leitura dos tabelas .csv**

In [ ]:
tabela_cliente = pd.read_csv('/content/1_cliente.csv')
tabela_localizacao = pd.read_csv('/content/2_geolocalizacao.csv')
tabela_pedido = pd.read_csv('/content/3_pedido.csv')
tabela_pagamento = pd.read_csv('/content/4_pagamento.csv')
tabela_review = pd.read_csv('/content/5_review_clientes.csv')
tabela_entrega_pedido = pd.read_csv('/content/6_entrega_pedido.csv')
tabela_descricao_produto = pd.read_csv('/content/7_descricao_produto.csv')
tabela_vendedor = pd.read_csv('/content/8_vendedor.csv')
tabela_categoria_traduzido = pd.read_csv('/content/9_categoria_traduzido.csv')

**Checagem dos 5 primeiros elementos de cada tabela**:

1- Use .head() para visualizar as primeiras linhas de cada uma das tabelas:

**Início do processo de união das 9 tabelas disponibilizadas com a finalidade de produzir uma tabela resultante que possua os elementos mais importantes para a determinação do review_score. No primero merge realizado, unimos a tabela de clientes com as respectivas entregas dos pedidos usando o código individual de cada consumidor como parâmetro.**

Conhecendo o "merge"

In [ ]:
pd.merge(tabela_cliente, tabela_entrega_pedido, 
         on=["customer_id"], how="inner")

**Processo de união de cada tabela com a próxima**

**1 - (Clientes, Entregas)**

**2 - (1, Pedidos)**

**3 - (2, Pagamentos)**

**4 - (3, Review)**

**5 - (4, Vendedor)**

In [ ]:
dados = tabela_cliente.merge(tabela_entrega_pedido).merge(tabela_pedido).merge(tabela_pagamento).merge(tabela_review).merge(tabela_vendedor)

Visualizando a tabela resultante

In [ ]:
#dados.head()

**Segunda filtragem consiste em remover elementos que não possuem
relação com a variável review_score**

In [ ]:
df_= dados.copy()

In [ ]:
dados.columns.tolist()

In [ ]:
dataframe = dados.drop(columns=["customer_unique_id","customer_city",
                          "customer_state", "order_status", 
                          "order_purchase_timestamp", 
                          "order_approved_at", 
                          "order_delivered_carrier_date", 
                          "order_delivered_customer_date",
                          "order_estimated_delivery_date", 
                          "shipping_limit_date", 
                          "review_creation_date", 
                          "review_answer_timestamp", 
                          "seller_city", 
                          "seller_state", 
                          "review_comment_title", 
                          "review_comment_message"])


**Tabela Resultante após a remoção de atributos não prioritários para o nível de
satisfação dos clientes**

2- Use shape para verificar as dimensões da tabela resultante.

In [ ]:
dataframe

In [ ]:
dataframe.shape

(117329, 15)

#vetores
Criando uma estrutura para evitar código longo:

---



**Inserindo cada atributo numérico da tabela resultante em um vetor para melhor manipulação dos dados**

vetor = np.array(dataframe.coluna)

In [ ]:
"""vetores = []
for coluna in dataframe:
  vetores.append(np.array(dataframe[coluna]))

cliente,cepcliente, pedido,\
idpedido, produto, vendedor,\
preco_produto, frete, parcelas,\
tipopagamento, prestações, valorfinal,\
review, score, cepvendedor = (vetor for vetor in vetores)"""


In [ ]:
for coluna in dataframe:
  dataframe[coluna] = (np.array(dataframe[coluna]))

In [ ]:
cepcliente

**Definindo um novo dataframe vazio**

In [ ]:
"""df = pd.DataFrame()
df"""

""


In [ ]:
"""colunas = ["cliente","cepcliente", "pedido",
"idpedido", "produto", "vendedor",
"preco_produto", "frete", "parcelas",
"tipopagamento", "prestações", "valorfinal",
"review", "score", "cepvendedor"]"""

In [ ]:
dataframe.columns = ["cliente","cepcliente", "pedido",
"idpedido", "produto", "vendedor",
"preco_produto", "frete", "parcelas",
"tipopagamento", "prestações", "valorfinal",
"review", "score", "cepvendedor"]

**Definindo as colunas do novo dataframe e atribuindo para cada coluna, seu respectivo vetor de dados registrado anteriormente.**

In [ ]:
#df = pd.DataFrame(columns =colunas)


In [ ]:
"""for col in df:
    df[col] = (vetor for vetor in vetores)"""

In [ ]:
"""df = pd.DataFrame(columns =colunas)
for coluna in df.columns:
    coluna = (vetor for vetor in vetores)

df.Cliente = vetor_cliente
df.CEP_Cliente = vetor_cepcliente
df.Pedido = vetor_pedido
df.id_Pedido = vetor_idpedido
df.Produto = vetor_produto
df.Vendedor = vetor_vendedor
df.Preco_produto = vetor_preco_produto
df.Frete = vetor_frete
df.Parcela = vetor_parcela
df.Tipo_pagamento = vetor_tipopagamento
df.Installments = vetor_pay
df.Valor_total = vetor_valorfinal
df.ID_Review = vetor_review
df.CEP_Vendedor = vetor_cepvendedor
df.Score = vetor_score
df"""

**Impressão da coluna de clientes.**

In [ ]:
type(dataframe.cliente)
#verificar se o original continua como series

pandas.core.series.Series

**Informações do dataframe**

**Atributos, elementos não nulos, Tipo das variáveis da coluna**

In [ ]:
dataframe.info()

**Agrupando os elementos do dataframe por consumidor**

In [ ]:
dataframe.groupby(by='cliente').size()

#**Importando os métodos de árvore de decisão**

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics

**Para simplificar os dados, removemos todos os elementos não-numéricos para que o modelo seja capaz de realizar a execução. Solução encontrada para simplificar os atributos do tipo "Objeto" para tipos numéricos.**

In [ ]:
df['Cliente'] = df['Cliente'].str.replace(r'\D', '')
df['Pedido'] = df['Pedido'].str.replace(r'\D', '')
df['Produto'] = df['Produto'].str.replace(r'\D', '')
df['Vendedor'] = df['Vendedor'].str.replace(r'\D', '')
df['ID_Review'] = df['ID_Review'].str.replace(r'\D', '')
df

**Realizamos o procedimento de remoção dos elementos não-numéricos para todas as colunas do tipo objeto com exceção do tipo de pagamento pois o tipo de pagamento se resume a poucas opções. Dessa forma, usamos a função get_dummies apenas para o tipo de pagamento**

**Portanto, a coluna Tipo_pagamento se divide em quatro colunas com lógica booleana. As novas colunas são: Tipo_pagamento_boleto, 	Tipo_pagamento_credit_card, Tipo_pagamento_debit_card, Tipo_pagamento_voucher**

In [ ]:
result_df = pd.get_dummies(df, columns=["Tipo_pagamento"])


**Resultado final do dataframe**

In [ ]:
result_df

,Cliente,CEP_Cliente,Pedido,id_Pedido,Produto,Vendedor,Preco_produto,Frete,Parcela,Installments,Valor_total,ID_Review,CEP_Vendedor,Score,Tipo_pagamento_boleto,Tipo_pagamento_credit_card,Tipo_pagamento_debit_card,Tipo_pagamento_voucher
0,068999211881720087,14409,0071050849957707322971,1,9516079379936978101698,76714480069693656010,124.99,21.88,1,2,146.87,88852460269121365930,8577,4,0,1,0,0
1,4730028858608712037403,1151,205967144362972992,1,0766896614945862518487,76714480069693656010,139.94,17.79,1,7,157.73,56695761864734270698487,8577,5,0,1,0,0
2,260275510952946358123,8775,9516709235944631127306,1,564744977148034751092,76714480069693656010,149.94,23.36,1,1,173.30,0598013162266672875,8577,5,0,1,0,0
3,406427971077955491,65075,9974373861057480116,1,8094076568610375,76714480069693656010,199.98,115.77,1,6,315.75,2518803238635645934852,8577,1,0,1,0,0
4,827607252254128462738664,90010,631862016861141129,1,8609375091115750004,76714480069693656010,129.60,16.61,1,1,146.21,84467176677541425367905,8577,4,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117324,1075167500095617,39920,49462463110524497277085909,1,242136271129618259334261,9140758211403593580567204,49.90,22.28,1,3,72.18,70902082652897569110,87015,5,0,1,0,0
117325,0671403498200452602,7231,7331435643056154391,1,51514648249690076,414909055710300356,730.00,45.98,1,10,775.98,274388990740389448855,9861,3,0,1,0,0
117326,908572770808078332363658,76400,76905458869336527267,1,6723658311543806891676,377183347821535223225,135.98,88.46,1,6,224.44,601019591910776193298,83015,5,0,1,0,0
117327,108322995705926071768,37610,5162460589765251860206,1,98477838202378681,5104862382080742,167.99,31.93,1,5,167.79,60328738426975435430539769,12914,1,0,1,0,0


**Criação de um dataframe reserva para possíveis conclusões**

In [ ]:
reserva = result_df.copy()
reserva

**Eliminando todas as linhas com nível 4 ou nível 5 de satisfação. Dessa forma, temos um dataframe com todos os dados, e um apenas com dados classificados com nível 3,2 ou 1 ou seja, mediano, ruim ou péssimo. (elementos que apresentam nível de insatisfação interessante para análise)**

In [ ]:
reserva = reserva.drop(reserva[reserva.Score > 3].index)
reserva


**Processo de separação de treine/teste**

**Proporções estabelecidas:**

**70% treino**

**30% teste**

In [ ]:
y = result_df['Score']

In [ ]:
X = result_df.drop('Score', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

**Número de amostras para cada processo**

In [ ]:
X_train.shape, X_test.shape

((82130, 17), (35199, 17))

**Número de targets para cada processo**

In [ ]:
y_train.shape, y_test.shape

((82130,), (35199,))

**Criação do classifcador**

In [ ]:
cls = DecisionTreeClassifier()

**Treinamento**

In [ ]:
cls = cls.fit(X_train, y_train)

**Vetor com as importancias de cada atributo para a determinação do review_score**

In [ ]:
cls.feature_importances_

array([0.10822897, 0.11241319, 0.10474687, 0.00635806, 0.09510356,
       0.0750454 , 0.0773494 , 0.08636357, 0.00276909, 0.03842658,
       0.08727398, 0.11130009, 0.07212105, 0.00813314, 0.00919786,
       0.00199643, 0.00317274])

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, 
export_graphviz, export_text


In [ ]:
plt.rcParams["figure.figsize"] = (40, 20)


In [ ]:
"""plt.figure()
plot_tree(cls, filled=False)
plt.show()"""

'plt.figure()\nplot_tree(cls, filled=False)\nplt.show()'

In [ ]:
df.head()

**Para tornar o modelo mais visual, criamos um laço para a impressão dos pesos de cada atributo para determinar o score**

In [ ]:
for feature, importancia in zip(result_df.columns, cls.feature_importances_):
    print("{}:{:.1f}%".format(feature,((importancia*100))))


Cliente:10.8%
CEP_Cliente:11.2%
Pedido:10.5%
id_Pedido:0.6%
Produto:9.5%
Vendedor:7.5%
Preco_produto:7.7%
Frete:8.6%
Parcela:0.3%
Installments:3.8%
Valor_total:8.7%
ID_Review:11.1%
CEP_Vendedor:7.2%
Score:0.8%
Tipo_pagamento_boleto:0.9%
Tipo_pagamento_credit_card:0.2%
Tipo_pagamento_debit_card:0.3%


**Vetor com predições para checagem do aprendizado**

In [ ]:
result = cls.predict(X_test)
result

array([3, 4, 5, ..., 4, 3, 5])

**Representação das métricas de precisão e médias do modelo**

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test,result))

              precision    recall  f1-score   support

           1       0.26      0.27      0.26      4472
           2       0.14      0.14      0.14      1237
           3       0.15      0.16      0.16      3026
           4       0.24      0.25      0.25      6694
           5       0.61      0.59      0.60     19770

    accuracy                           0.43     35199
   macro avg       0.28      0.28      0.28     35199
weighted avg       0.44      0.43      0.44     35199



**Precisão total**

In [ ]:
from sklearn.model_selection import cross_val_score
allScores = cross_val_score(cls, X_train, y_train , cv=10)
allScores.mean()

0.43187629368075003

**Treinamento utilizando o dataframe reserva. (apenas com os níveis de satisfação abaixo da média, score <=3)** 

**Split do dataframe em treino e teste (70%, 30%, respectivamente)**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reserva.drop('Score', axis=1), 
                                                    reserva['Score'], 
                                                    test_size=0.2)

**Quantidade de amostras do treino**

In [ ]:
X_train.shape, X_test.shape


**Classificador**

In [ ]:
clf = DecisionTreeClassifier()

**Treino**

In [ ]:
clf = clf.fit(X_train, y_train)


**Importancia de cada atributo para determinar o nível de satisfação dos consumidores**

In [ ]:
clf.feature_importances_

array([0.10941927, 0.10507044, 0.10698141, 0.00971772, 0.08788176,
       0.08519155, 0.08140284, 0.08699867, 0.00351186, 0.03624902,
       0.0913872 , 0.1065729 , 0.07112704, 0.00711295, 0.00700647,
       0.0023347 , 0.00203419])

In [ ]:
for feature, importancia in zip(reserva.columns, clf.feature_importances_):
    print("{}:{:.1f}%".format(feature,((importancia*100))))
    

Cliente:10.9%
CEP_Cliente:10.5%
Pedido:10.7%
id_Pedido:1.0%
Produto:8.8%
Vendedor:8.5%
Preco_produto:8.1%
Frete:8.7%
Parcela:0.4%
Installments:3.6%
Valor_total:9.1%
ID_Review:10.7%
CEP_Vendedor:7.1%
Score:0.7%
Tipo_pagamento_boleto:0.7%
Tipo_pagamento_credit_card:0.2%
Tipo_pagamento_debit_card:0.2%


# ANÁLISE DOS DADOS PROCESSADOS

**Com o objetivo de aumentar o review_score desse marketplace, utilizamos o algoritmo de árvore de decisão para encontrar elementos que influenciam diretamente a aceitação e satisfação dos clientes, analisando desde o processo de despacho do produto, até a qualidade do atendimento e do produto final. Dessa forma, utilizamos estratégias de filtragem. Inicialmente eliminamos os elementos que, evidentemente, não apresentavam influencia sobre a nota registrada pelo cliente. Após esse passo inicial, simplificamos o tipo dos dados para facilitar o processamento do dataframe pelo algoritmo. Após o processo de preparar os dados para o aprendizado registramos os seguintes valores:**

**Considerando todos as notas (Ótimo, bom, mediano, ruim, péssimo) :**

Cliente:9.7%

CEP_Cliente:11.6%

Pedido:10.2%

id_Pedido:0.4%

Produto:8.6%

Vendedor:8.8%

Preco_produto:8.5%

Frete:8.5%

Parcela:0.1%

Installments:3.7%

Valor_total:10.2%

ID_Review:10.3%

CEP_Vendedor:7.6%

Score:0.8%

Tipo_pagamento_boleto:0.6%

Tipo_pagamento_credit_card:0.2%

Tipo_pagamento_debit_card:0.1%


------------------------------------


---------------------
**Portanto, com base nos resultados obtidos, o dono do marketplace deve se atentar aos seguintes parâmetros de sua logística:**

**1 - Relação entre CEP_Cliente, Frete e CEP_Vendedor**

**Esses atributos apresentaram importancia direta na nota registrada pelo cliente, dessa forma, algumas razões devem ser consideradas:**

**-Problemas com a entrega.**

**-Qualidade da entrega(tempo de entrega,comprometimento do produto durante o processo de transporte).**

**-Alto preço do frete para determinadas regiões.**

-----------------------------
**2 - Produto**

**O atributo produto apresentou importancia direta na insatisfação dos clientes. Portanto, deve-se considerar:**

**-Qualidade ruim de determinados produtos.**

**-O produto entregue não apresentar as caracteristicas do produto anunciado.**

**-Entrega de produtos errados, problema de logística.**

---------------------------------

**3 - Vendedor**

**O atributo Vendedor apresentou importancia direta na insatisfação dos clientes. Portanto, deve-se considerar:**

**-Qualidade duvidosa do atendimento por parte do vendedor.**

**-Erro do vendedor em alguma etapa do processo em específico.**

--------------------------


# **CONCLUSÃO**


**Portanto, é possível concluir que para o marketplace aumentar seu score, analisar problemas logísticos por parte do processo de transporte, aspectos do produto e os vendedores são os fatores mais importantes para entender o que se passa na empresa e solucionar o problema que está gerando insatisfação dos clientes desse e-commerce.**





